# This exercise was done on Google Colab with runtime as GPU.

In [32]:
!pip install keras-tuner

In [33]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [34]:
print(tf.__version__)

2.4.0


In [35]:
fashion_mnist=keras.datasets.fashion_mnist

In [36]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [37]:
# we will scale down rhe data between 0 to 1 by diving by 255
train_images=train_images/255.0
test_images=test_images/255.0

In [38]:
#print(train_images[0])
print(train_images[0].shape)

(28, 28)


In [39]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [40]:
train_images.shape

(60000, 28, 28, 1)

In [41]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [42]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [56]:
tuner_search = RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="output",project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [57]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 00m 18s]
val_accuracy: 0.10499999672174454

Best val_accuracy So Far: 0.9111666679382324
Total elapsed time: 00h 01m 13s
INFO:tensorflow:Oracle triggered exit


In [58]:
model = tuner_search.get_best_models(num_models=1)[0]

In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        76864     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,052,106
Trainable params: 3,052,106
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)
# when we say intial_epeoch it means we have already train the model till 3 hence it starts from 4 to 10

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1720 - accuracy: 0.9365 - val_loss: 0.2441 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1135 - accuracy: 0.9565 - val_loss: 0.2700 - val_accuracy: 0.9098
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0771 - accuracy: 0.9718 - val_loss: 0.3382 - val_accuracy: 0.9120
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0562 - accuracy: 0.9797 - val_loss: 0.3758 - val_accuracy: 0.9117
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0396 - accuracy: 0.9864 - val_loss: 0.3679 - val_accuracy: 0.9113
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0276 - accuracy: 0.9899 - val_loss: 0.4450 - val_accuracy: 0.9165
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0236 - accuracy: 0.9920 - val_loss: 0.4854 - val_accuracy